In [ ]:


from PIL import Image
import os

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import matplotlib.pyplot as plt


In [12]:
#defining generator model
class Generator(nn.Module):
  def __init__(self):
    super(Generator,self).__init__()
    #calling nn module parameteralized constructor to initialize its internak state
    self.model=nn.Sequential(
        #creating 2d convulation with 1 input channel and 64 output channels(64 filters)
        nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(64),
        nn.Conv2d(64,128,kernel_size=3,stride=2,padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(128),
        nn.Conv2d(256,512,kernel_size=3,stride=2,padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(512),
        nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(256),
        nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(128),
        nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(64),
        nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
        nn.Tanh()
    )
  def forward(self,x):
    return self.model(x)

In [13]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.model=nn.Sequential(
      nn.Conv2d(3,64,kernel_size=3,stride=2,padding=1),
      nn.LeakyReLU(0.2),
      nn.Dropout(0.25),
      #randomly drop 25% neurons to avoid overfitting
      nn.Conv2d(64,128,kernel_size=3,stride=2,padding=1),
      nn.BatchNorm2d(128),
      nn.Dropout(0.25),

      nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
      nn.LeakyReLU(0.2),
       nn.BatchNorm2d(256),
     nn.Dropout(0.25),

     nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
     nn.LeakyReLU(0.2),
     nn.BatchNorm2d(512),
     nn.Dropout(0.25),

     nn.Flatten(),
     nn.Linear(512 * 16 * 16, 1),
     nn.Sigmoid()

      #flattening to convert the multidimensional tensor
      #to single dimensional tensor so that it can be taken as input for classification task done by neural descriminator



    )
  def forward(self,x):
    return self.model(x)

In [14]:
#defining loss functions and optimizers
criterion = nn.BCELoss()

def get_optimizer(model, lr=0.0002, beta1=0.5):
    return optim.Adam(model.parameters(), lr=lr, betas=(beta1, 0.999))
#loss function is Binary cross entropy loss and optimizer is Adam
def load_image(image_path, target_size=(256, 256)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(target_size)
    image = np.array(image) / 127.5 - 1.0  # Normalize to [-1, 1]
    return torch.tensor(image, dtype=torch.float).permute(2, 0, 1).unsqueeze(0)

def load_data(sar_dir, optical_dir):
    sar_images = []
    optical_images = []

    for filename in os.listdir(sar_dir):
        sar_image = load_image(os.path.join(sar_dir, filename), target_size=(256, 256))
        optical_image = load_image(os.path.join(optical_dir, filename), target_size=(256, 256))

        sar_images.append(sar_image[:, 0:1])  # Grayscale
        optical_images.append(optical_image)

    return torch.cat(sar_images), torch.cat(optical_images)

